In [ ]:
# This jupyter note book helps play around with queries manually

In [18]:
from dotenv import load_dotenv
import os
from csdash import dbaccess
from rich import print
import time 
import pandas as pd

In [19]:
# load the dependencies and check instaleld correctly 
load_dotenv()

es = dbaccess.connect_cogstack()
user = os.environ.get("ELASTIC_API_ID")
print(user)

TbF70ZQBjIvyPy2Q2836

In [46]:

index = "gstt_clinical_geneworks_documents_20230404"

aggregatesquery = {
  "size":0,
  "aggs":{
    "unique_doc_cat":{
      "terms":{
        "field":"document_Category",
        "size": 10000
      }
    },
    "unique_vals_doccontenttypes":{
      "terms":{
        "field":"document_ContentType",
        "size":10000
      }
    },
    "unique_doc_Genetics_area":{
      "terms":{
        "field":"document_GeneticsArea",
        "size":10000
      }
    }
  }
}


In [47]:
response = es.search(index=index, body=aggregatesquery)

In [48]:
print(response["aggregations"])

{
    'unique_doc_Genetics_area': {
        'doc_count_error_upper_bound': 0,
        'sum_other_doc_count': 0,
        'buckets': [
            {'key': 'Cytogenetics', 'doc_count': 157836},
            {'key': 'Molecular Genetics', 'doc_count': 50200},
            {'key': 'Biochemistry', 'doc_count': 28507}
        ]
    },
    'unique_vals_doccontenttypes': {
        'doc_count_error_upper_bound': 0,
        'sum_other_doc_count': 0,
        'buckets': [
            {'key': 'application/msword', 'doc_count': 312128},
            {'key': 'application/pdf', 'doc_count': 153095},
            {
                'key': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
                'doc_count': 145610
            },
            {'key': 'text/plain; charset=windows-1252', 'doc_count': 36293},
            {'key': 'application/vnd.wordperfect; version=5.1', 'doc_count': 21553},
            {'key': 'text/plain; charset=iso-8859-1', 'doc_count': 2282},
            {'key': 'application/vnd.ms-outlook', 'doc_count': 687},
            {'key': 'image/jpeg', 'doc_count': 244},
            {'key': 'image/tiff', 'doc_count': 74},
            {'key': 'application/rtf', 'doc_count': 26},
            {'key': 'image/png', 'doc_count': 17},
            {'key': 'message/rfc822', 'doc_count': 15},
            {'key': 'text/html; charset=windows-1250', 'doc_count': 4},
            {'key': 'application/octet-stream', 'doc_count': 2},
            {'key': 'text/html; charset=ibm866', 'doc_count': 2},
            {'key': 'text/html; charset=iso-8859-7', 'doc_count': 2},
            {'key': 'text/html; charset=x-maccyrillic', 'doc_count': 2},
            {'key': 'text/html; charset=ibm855', 'doc_count': 1},
            {'key': 'text/html; charset=shift_jis', 'doc_count': 1},
            {'key': 'text/html; charset=us-ascii', 'doc_count': 1},
            {'key': 'text/html; charset=utf-8', 'doc_count': 1},
            {'key': 'text/html; charset=windows-1252', 'doc_count': 1}
        ]
    },
    'unique_doc_cat': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': []}
}

In [49]:
query4 = {
  "_source": [
    "document_Category",
    "document_GeneticsArea",
    "document_CreatedWhen"
    "document_UpdatedWhen",
    "document_Name",
    "document_Content",
    "document_Notes",
    "document_SourceId",
    "document_TrustId",
    "document_Type"
    "document_SpecimenNumber"
    "patient_Age",
    "patient_HospitalNumber",
    "patient_SourceId",
    "patient_NhsNumber",
    "patient_FirstName",
    "patient_MiddleName",
    "patient_LastName",
    "patient_Gender",
    "patient_DateOfBirth",
    "patient_Ethnicity",
    "patient_EpicId"
  ],
  "query": {
    "bool": {
      "should":[
        {"match_phrase":{"document_Content":{
          "query": "BRCA"
        }}},
        {"match_phrase":{"document_Content":{
          "query": "breast cancer gene"
        }}},
        {"match_phrase":{"document_Content":{
          "query": "breast cancer gene variant"
        }}},
        {"match_phrase":{"document_Content":{
          "query": "brca1"
        }}},
        {"match_phrase":{"document_Content":{
          "query": "brca2"
        }}},
        {"match_phrase":{"document_Content":{
          "query": "brca mutation"
        }}},
        ],
      "minimum_should_match":1,
    }
   }
}

In [50]:
response2 = es.search(index=index, body=query4,scroll="3m")


In [51]:
print(response2["hits"]["total"]["value"])

25251

In [55]:
def get_all_hits(index_name,query,size=10000):
    
    result = es.search(
        index = index_name,
        body = query,
        scroll = "5m",
        size = size
    )
    
    scroll_id = result["_scroll_id"]
    hits = result["hits"]["hits"]
    
    while len(result["hits"]["hits"]):
        result = es.scroll(scroll_id = scroll_id, scroll = "5m")
        hits.extend(result["hits"]["hits"])
        print(f"retrieved {len(hits)} hits thus far")
        time.sleep(0.1)
    
    es.clear_scroll(scroll_id = scroll_id)
    df = pd.json_normalize(hits)
    return df  
        
    

In [57]:
df = get_all_hits(index_name=index,query=query4,)

/tmp/ipykernel_16313/4261326484.py:6: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  result = es.search(


retrieved 20000 hits thus far

retrieved 25251 hits thus far

retrieved 25251 hits thus far

In [60]:
from elastic_utils import ElasticsearchSession

ModuleNotFoundError: No module named 'elastic_utils'

In [58]:
df.shape

(25251, 18)

In [59]:
df.describe

<bound method NDFrame.describe of                                            _index         _id     _score  \
0      gstt_clinical_geneworks_documents_20230404  gwr_304914  41.634590   
1      gstt_clinical_geneworks_documents_20230404   gwd_29717  40.377316   
2      gstt_clinical_geneworks_documents_20230404   gwd_29807  40.164360   
3      gstt_clinical_geneworks_documents_20230404  gwd_354164  40.139870   
4      gstt_clinical_geneworks_documents_20230404    gwd_5728  40.137283   
...                                           ...         ...        ...   
25246  gstt_clinical_geneworks_documents_20230404   gwd_78810   0.451600   
25247  gstt_clinical_geneworks_documents_20230404  gwd_201953   0.369654   
25248  gstt_clinical_geneworks_documents_20230404   gwd_74926   0.368222   
25249  gstt_clinical_geneworks_documents_20230404  gwd_326065   0.299470   
25250  gstt_clinical_geneworks_documents_20230404  gwd_338931   0.297297   

       _source.document_TrustId _source.document_Name

In [54]:
df.to_excel("test.xlsx")